In [2]:
import sys
import os
if not any(path.endswith('textbook') for path in sys.path):
    sys.path.append(os.path.abspath('../../..'))
from textbook_utils import *

(ch:pa)=
# Case Study: How Accurate are Air Quality Measurements?

California is prone to wildfires, so much so that its
residents (like the authors of this book) sometimes say
that California is "always on fire".
In 2020, forty separate fires covered the state in smoke,
caused thousands of people to evacuate, and caused more than twelve billion
dollars in damages ({numref}`fig:ca-fires`).

```{figure} figures/ca-fires.jpeg
---
name: fig:ca-fires
alt: ca-fires
---
Satellite image from August 2020 showing smoke covering California. (Image from [Wikipedia](https://en.wikipedia.org/wiki/File:California_on_fire_ESA22177856.jpeg) licensed under CC BY-SA 3.0 IGO.)
```

In places like California, people use air quality measurements to learn what kinds of protective measures they need to take.
Depending on conditions, people may wish to wear a mask, use air filters, or avoid going outside altogether.

In the United States, One important source of air measurements is the Air Quality System (AQS), run by the US government {cite}`usepaAir2013`.
The AQS places high-quality sensors at locations across the US and makes this data available to the public.
These sensors are carefully calibrated to strict standards---in fact, the AQS sensors are generally seen as the gold standard for accuracy.
However, they have a few downsides.
These sensors are expensive: typically between \$15,000  and  \$40,000 USD each.
This means that there are fewer sensors, and these sensors are further apart.
A person living far away from a sensor might not be able to use these measurements for their personal use.
Also, these sensors do not provide real-time data. Since the data undergo additional calibration, the sensors only release hourly averages with a time lag of one to two hours. In essence, the AQS sensors are accurate but not timely.

In contrast to the AQS, [PurpleAir][pa] sensors sell for about \$250 and can be easily installed at home.
With the lower price point, thousands of people across the US have purchased these sensors for personal use. The sensors can connect to a home WiFi network so the air quality can be easily monitored.
These sensors can also report data back to the PurpleAir company.
In 2020, thousands of owners of PurpleAir sensors made publicly available their sensors' measurements of air quality.
Compared to the AQS sensors, PurpleAir sensors are more timely. They make a measurement every two minutes rather than every hour.
Since there are more deployed PurpleAir sensors, more people live close enough to a sensor to make use of the data.
However, PurpleAir sensors are less accurate.
To make the sensors affordable, PurpleAir uses a simpler method of counting particles in air that doesn't measure particle density.
This means that PurpleAir measurements can report that air quality is worse than it really is {cite}`hug2020`. In essence, PurpleAir sensors are timely but not accurate.

[pa]: https://www2.purpleair.com/

In this chapter we plan to use the AQS sensor measurements to improve the PurpleAir measurements. It's a big task, and we follow the analysis first developed by Karoline Barkjohn et al. from the US Environmental Protection Agency {cite}`barkjohnDevelopment2021`.
Her group's work was so successful that, as of this writing, the official US government maps, like the AirNow Fire and Smoke map, includes both AQS and PurpleAir sensors, and applies the correction Barkjohn developed to the PurpleAir sensors.

Our work follows the data science life cycle, beginning with considering the question and the scope of the available data. Much of our effort is spent cleaning and wrangling the data into shape for analysis, but we also carry out an exploratory data analysis and build a model. 